##PERGUNTAS QUE O MVP DEVE RESPONDER

###Padrões gerais de gasto
- Qual é o valor total gasto pela CEAP no período analisado?
- Qual é o gasto médio mensal por parlamentar?
- Como os gastos se distribuem ao longo dos meses do ano?
###Análise por parlamentar
- Quais parlamentares apresentam os maiores volumes de despesas?
- Existe grande variação de gastos entre parlamentares?
###Análise por tipo de despesa
- Quais tipos de despesa concentram a maior parte dos gastos da CEAP?
-- Ex: passagens aéreas, combustíveis, divulgação, hospedagem
###Análise por partido político
- Qual é o gasto médio por partido político?
- A composição de gastos varia entre partidos?
###Análise geográfica (UF)
- Como os gastos se distribuem entre as UFs?
###Análise temporal
- Há meses com picos atípicos de despesas?
- Determinados tipos de despesa concentram-se em períodos específicos do ano?


In [0]:
%sql
USE CATALOG mvp_ed_ceap;
USE SCHEMA layer_gold;

###Padrões gerais de gasto

In [0]:
%sql
-- Qual é o valor total gasto pela CEAP no período analisado?
SELECT
  SUM(valor_documento) AS gasto_total_ceap
FROM fato_despesa_ceap;


In [0]:
%sql
-- Qual é o gasto médio mensal por parlamentar?
SELECT
  p.nome_parlamentar,
  AVG(f.valor_documento) AS gasto_medio
FROM fato_despesa_ceap f
JOIN dim_parlamentar p
  ON f.id_parlamentar = p.id_parlamentar
GROUP BY p.nome_parlamentar
ORDER BY gasto_medio DESC
Limit(10); -- Adicionado limite para reduzir o output no Git

In [0]:
%sql
--Como os gastos se distribuem ao longo dos meses do ano?
SELECT
  t.ano,
  t.mes,
  SUM(f.valor_documento) AS total_mes
FROM fato_despesa_ceap f
JOIN dim_tempo t
  ON f.id_tempo = t.id_tempo
GROUP BY t.ano, t.mes
ORDER BY t.ano, t.mes;


---
###Análise por parlamentar

In [0]:
%sql
--Quais parlamentares apresentam os maiores volumes de despesas?
SELECT
  p.nome_parlamentar,
  SUM(f.valor_documento) AS total_gasto
FROM fato_despesa_ceap f
JOIN dim_parlamentar p
  ON f.id_parlamentar = p.id_parlamentar
GROUP BY p.nome_parlamentar
ORDER BY total_gasto DESC
LIMIT 10;


In [0]:
%sql
--Existe grande variação de gastos entre parlamentares?
SELECT
  MIN(total_gasto) AS menor_gasto,
  MAX(total_gasto) AS maior_gasto,
  AVG(total_gasto) AS gasto_medio
FROM (
  SELECT
    id_parlamentar,
    SUM(valor_documento) AS total_gasto
  FROM fato_despesa_ceap
  GROUP BY id_parlamentar
);


---
###Análise por tipo de despesa

In [0]:
%sql
--Quais tipos de despesa concentram a maior parte dos gastos?
SELECT
  d.descricao_tipo_despesa,
  SUM(f.valor_documento) AS total_gasto
FROM fato_despesa_ceap f
JOIN dim_tipo_despesa d
  ON f.id_tipo_despesa = d.id_tipo_despesa
GROUP BY d.descricao_tipo_despesa
ORDER BY total_gasto DESC;


---
###Análise por partido político

In [0]:
%sql
--Qual é o gasto médio por partido político?
SELECT
  p.sigla_partido,
  AVG(f.valor_documento) AS gasto_medio
FROM fato_despesa_ceap f
JOIN dim_partido p
  ON f.id_partido = p.id_partido
GROUP BY p.sigla_partido
ORDER BY gasto_medio DESC;


In [0]:
%sql
--A composição de gastos varia entre partidos?
SELECT
  p.sigla_partido,
  d.descricao_tipo_despesa,
  SUM(f.valor_documento) AS total_gasto
FROM fato_despesa_ceap f
JOIN dim_partido p
  ON f.id_partido = p.id_partido
JOIN dim_tipo_despesa d
  ON f.id_tipo_despesa = d.id_tipo_despesa
GROUP BY p.sigla_partido, d.descricao_tipo_despesa
ORDER BY p.sigla_partido, total_gasto DESC
Limit(30); -- Adicionado limite para reduzir o output no Git


---
###Análise geográfica (UF)

In [0]:
%sql
--Como os gastos se distribuem entre as UFs?
SELECT
  u.sigla_uf,
  SUM(f.valor_documento) AS total_gasto
FROM fato_despesa_ceap f
JOIN dim_uf u
  ON f.id_uf = u.id_uf
GROUP BY u.sigla_uf
ORDER BY total_gasto DESC;


---
###Análise temporal

In [0]:
%sql
--Há meses com picos atípicos de despesas?
SELECT
  t.ano,
  t.mes,
  SUM(f.valor_documento) AS total_mes
FROM fato_despesa_ceap f
JOIN dim_tempo t
  ON f.id_tempo = t.id_tempo
GROUP BY t.ano, t.mes
ORDER BY total_mes DESC;

